## EGGS Performance Results

In [1]:
import os
from itertools import product

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cufflinks as cf
import plotly.offline

from IPython.display import display, HTML
from ipywidgets import interactive_output, HBox, VBox, Layout 
from ipywidgets import Dropdown, Checkbox, ToggleButtons, SelectionRangeSlider, SelectionSlider
from ipywidgets import SelectMultiple
import ipywidgets as widgets

cf.go_offline()
pd.set_option('display.max_columns', 100)

data_dir = '../../'
df = pd.read_csv(os.path.join(data_dir, 'results.csv'))
df.head()

,dataset,fold,rs,base_estimator,feature_type,test_type,sgl_method,sgl_stacks,pgm,auc,ap
0,youtube,0,1,lr,limited,full,None,0,None,0.783103,0.453376
1,youtube,0,1,lr,limited,full,None,0,mrf,0.799473,0.471791
2,youtube,0,1,lr,limited,full,holdout,1,None,0.762531,0.465430
3,youtube,0,1,lr,limited,full,holdout,1,mrf,0.795710,0.488608
4,youtube,0,1,lr,limited,full,holdout,2,None,0.740609,0.446908


In [2]:
# plot options
metric = ToggleButtons(description='Metric', options=['auc', 'ap'], value='auc')
errorbar = Checkbox(description='Error bars', value=False)

# experiment options
dataset = Dropdown(description='Dataset', options=['youtube', 'twitter', 'soundcloud'], value='youtube')
feature_type = ToggleButtons(description='Feature set', options=['full', 'limited'], value='full')
test_type = ToggleButtons(description='Test', options=['full', 'inductive'], value='full')
base_estimator = ToggleButtons(description='Criterion', options=['lr', 'lgb'], value='lr')
rs = SelectionSlider(description='rs', options=[1], value=1)

# hyperparameter options
sgl_method = SelectMultiple(description='SGL method', options=['None', 'holdout', 'cv'], value=('None', 'holdout'))
sgl_stacks = SelectionRangeSlider(description='SGL stacks', options=[0, 1, 2], index=(0, 2))
pgm = SelectMultiple(description='PGM', options=['None', 'psl', 'mrf'], value=('None', 'mrf'))

# create ui
box_1 = VBox([metric, rs, errorbar])
box_2 = VBox([dataset, feature_type, test_type, base_estimator])
box_3 = VBox([sgl_method, sgl_stacks, pgm])
ui = HBox([box_2, box_3, box_1])

# plot graphs
def f(metric, errorbar,
      dataset, feature_type, test_type, base_estimator, rs,
      sgl_method, sgl_stacks, pgm):
    
    # filter results
    temp = df.copy()
    temp = temp[temp['dataset'] == dataset]
    temp = temp[temp['feature_type'] == feature_type]
    temp = temp[temp['test_type'] == test_type]
    temp = temp[temp['base_estimator'] == base_estimator]
    temp = temp[temp['rs'] == rs]

    exp_df = temp.copy()
    
    temp = temp[temp['sgl_method'].isin(list(sgl_method))]
    temp = temp[(temp['sgl_stacks'] >= sgl_stacks[0]) & (temp['sgl_stacks'] <= sgl_stacks[1])]
    temp = temp[temp['pgm'].isin(list(pgm))]

    sgl_method_list = ['None', 'holdout', 'cv']
    sgl_stacks_list = [0, 1, 2]
    pgm_list = ['None', 'psl', 'mrf']

    tuples = product(*[pgm_list, sgl_method_list, sgl_stacks_list])

    res = []

    # get baseline
    baseline = None
    
    pgm_color_dict = {'None': 'blue', 'psl': 'orange', 'mrf': 'purple'}
    colors = []
    
    # baseline results
    if len(exp_df) > 0:
        baseline_df = exp_df[(exp_df['sgl_method'] == 'None') & (exp_df['sgl_stacks'] == 0) & (exp_df['pgm'] == 'None')]

    # EGGS results
    for pgm_i, method_i, stacks_i in tuples:

        # skip unused parameters
        if method_i == 'None' and stacks_i == 0 and pgm_i == 'None':
            continue

        eggs_df = temp[(temp['sgl_method'] == method_i) & (temp['sgl_stacks'] == stacks_i) & (temp['pgm'] == pgm_i)]

        if len(eggs_df) > 0:
            colors.append(pgm_color_dict[pgm_i])
            
            eggs_df = eggs_df.merge(baseline_df[['fold', metric]], on='fold', how='left')
            eggs_df['{}_diff'.format(metric)] = eggs_df['{}_x'.format(metric)] - eggs_df['{}_y'.format(metric)]
            result = {'key': '{}\n{}\n{}'.format(method_i, stacks_i, pgm_i)}
            result['{}_avg'.format(metric)] = eggs_df['{}_diff'.format(metric)].mean()
            result['{}_std'.format(metric)] = eggs_df['{}_diff'.format(metric)].std()
            res.append(result)

    # plot results
    if len(temp) > 0:

        color = {'auc': 'purple', 'ap': 'purple'}
        plot_df = pd.DataFrame(res)
        
        fig, ax0 = plt.subplots(figsize=(15, 5))

        yerr = plot_df['{}_std'.format(metric)] if errorbar else None
        ax0.bar(plot_df['key'], plot_df['{}_avg'.format(metric)], yerr=yerr, color=colors)
        ax0.set_ylabel(metric)
        ax0.set_title('Performance Difference against Baseline (Higher is better)')
        
        if baseline:
            ax0.axhline(baseline, linestyle='--', color='black', label='baseline')
            ax0.legend()
    
    else:
        plt.clf()

out = interactive_output(f, {'metric': metric, 'errorbar': errorbar,
                             'dataset': dataset, 'feature_type': feature_type, 'test_type': test_type,
                             'base_estimator': base_estimator, 'rs': rs,
                             'sgl_method': sgl_method, 'sgl_stacks': sgl_stacks, 'pgm': pgm})
display(ui, out)

Output()